# Example: 2D Simulation

## Set up Julia environment

In [2]:
import Pkg
Pkg.activate(mktempdir())

Pkg.add("CairoMakie")
Pkg.add("CSV")
Pkg.add("NPZ")
Pkg.add("PencilFFTs")
Pkg.develop(path=joinpath(@__DIR__, ".."))  # Load version of UltraDark in this repo

In [3]:
using UltraDark
using Test
using NPZ
using CairoMakie
using CSV

In [4]:
Threads.nthreads()

8

## Define initial conditions

Load a function that initialises solitons.

In [5]:
include(joinpath(@__DIR__, "init_soliton.jl"));

Define a 2D grid.

In [6]:
resol = 128
len = 10.0

grids = Grids((len, len, len/resol), (resol, resol, 1));

Add some solitons to the grid.  Strictly speaking, these aren't solitons in 2D, but they'll do for demonstration purposes.

In [7]:
mass = 10
position_1 = [-len/5, -len/5, 0]
position_2 = [-len/5, +len/5, 0]
velocity = [1, 0, 0]
phase_1 = 0
phase_2 = π
t0 = 0

add_soliton(grids, mass, position_1, velocity, phase_1, t0)
add_soliton(grids, mass, position_2, velocity, phase_2, t0)

## Set options

In [8]:
output_dir = joinpath(pwd(), "output", "2D")

output_times = 0:0.1:5
output_config = OutputConfig(output_dir, output_times);

In [9]:
options = Config.SimulationConfig();

## Run simulation

In [10]:
@time simulate!(grids, options, output_config)

  7.058545 seconds (9.16 M allocations: 914.631 MiB, 4.09% gc time, 88.05% compilation time: 7% of which was recompilation)


## Plot output

In [11]:
summary = CSV.File(joinpath(output_config.directory, "summary.csv"));

In [12]:
rho_init = npzread("$(output_config.directory)/rho_1.npy");
rho_last = npzread("$(output_config.directory)/rho_$(length(output_times)).npy");

In [13]:
# lims = extrema(rho_last)
rho_lims = (0, maximum(rho_last))

(0, 132.3429061865363)

In [14]:
fig_anim = Figure()
ax_anim = Axis(
    fig_anim[1, 1],
    aspect=DataAspect(),
)

hidedecorations!(ax_anim)

δ_lims = extrema(rho_last.-1)

cb_density = Colorbar(
    fig_anim[1, 2],
    limits=δ_lims,
    label = L"$\rho/\rho_{\text{crit}}$",
)

frame = Observable(1)

rho = lift(i -> npzread(joinpath(output_config.directory, "rho_$(i).npy"))[:, :, 1], frame)
δ = @lift($rho .- 1)

contourf!(
    ax_anim,
    grids.x[:, 1, 1], grids.y[1, :, 1],
    δ,
    colorrange = rho_lims,
)

Record(fig_anim, 1:length(output_times); framerate = 5) do f
    frame[] = f
end

VideoStream(Base.PipeEndpoint(RawFD(53) open, 0 bytes waiting), Process(`ffmpeg -y -loglevel quiet -threads 16 -framerate 5 -pixel_format rgb24 -f rawvideo -s:v 800x600 -r 5 -i pipe:0 -vf vflip -profile:v high422 -crf 20 -preset slow -c:v libx264 -pix_fmt yuv420p -an '/tmp/jl_MgLwwY/##video#227.mp4'`, ProcessRunning), CairoMakie.Screen{CairoMakie.IMAGE}(Scene (800px, 600px):
  0 Plots
  2 Child Scenes:
    ├ Scene (800px, 600px)
    └ Scene (800px, 600px), Cairo.CairoSurfaceImage{ColorTypes.ARGB32}(Ptr{Nothing} @0x000000000484ad70, 800.0, 600.0, ColorTypes.ARGB32[ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) … ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8); ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) … ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8); … ; ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) … ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8); ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) … ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8)]), Cairo.CairoContext(Ptr{Nothing} @0x000000000447dad0, Cairo.CairoSurfaceImage{ColorTypes.ARGB32}(Ptr{Nothing} @0x000000000484ad70, 800.0, 600.0, ColorTypes.ARGB32[ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) … ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8); ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) … ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8); … ; ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) … ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8); ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) … ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8) ARGB32(1.0N0f8,1.0N0f8,1.0N0f8,1.0N0f8)]), Ptr{Nothing} @0x000000000973a010), 1.0, 6, false, CairoMakie.ScreenConfig(1.0, 0.75, :best, false, false)), ColorTypes.RGB{FixedPointNumbers.N0f8}[RGB{N0f8}(1.0,1.0,1.0) RGB{N0f8}(1.0,1.0,1.0) … RGB{N0f8}(1.0,1.0,1.0) RGB{N0f8}(1.0,1.0,1.0); RGB{N0f8}(1.0,1.0,1.0) RGB{N0f8}(1.0,1.0,1.0) … RGB{N0f8}(1.0,1.0,1.0) RGB{N0f8}(1.0,1.0,1.0); … ; RGB{N0f8}(1.0,1.0,1.0) RGB{N0f8}(1.0,1.0,1.0) … RGB{N0f8}(1.0,1.0,1.0) RGB{N0f8}(1.0,1.0,1.0); RGB{N0f8}(1.0,1.0,1.0) RGB{N0f8}(1.0,1.0,1.0) … RGB{N0f8}(1.0,1.0,1.0) RGB{N0f8}(1.0,1.0,1.0)], "/tmp/jl_MgLwwY/##video#227.mp4", Makie.VideoStreamOptions("mp4", 5, 20, "high422", "yuv420p", nothing, "quiet", "pipe:0", true))